# Intro: Combine, Validate, and Calculate Accuracy
    Step 1: Validate and combine the 3 master files 
    
    Step 2: Calculate Accuracy for predictions and for drafts (will require simulations)

In [1]:
import pandas as pd
import numpy as np

# Combine 

### Load Data

In [2]:
df_1 = pd.read_csv('Master_Caleb.csv')

In [3]:
df_1.head(1)

,Unnamed: 0,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,2,Allen,C,Brooklyn Nets,2018-10-19,NYK,15,8.693321


In [4]:
df_2 = pd.read_csv('Master_Dhruv.csv')

In [5]:
df_2.head(1)

,Unnamed: 0,Player,Team,Position,Date,Opp,Fantasy_Score,predictions_2019
0,2,Al-Farouq Aminu,Portland Trail Blazers,F,20/10/2018,SAS,14,12.650065


In [6]:
df_3 = pd.read_csv('Master_wolf.csv')

In [7]:
df_3.head(1)

,Unnamed: 0,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,2,Aldridge,FALSE,San Antonio Spurs,2018-10-20,POR,12,23.908463


### Standardize 

In [8]:
df_1 = df_1.drop(columns='Unnamed: 0')

In [9]:
df_2 = df_2.drop(columns='Unnamed: 0')
df_2 = df_2[['Player','Position','Team','Date','Opp','Fantasy_Score','predictions_2019']]

In [10]:
df_3 = df_3.drop(columns='Unnamed: 0')
df_3['Position'] = np.where(df_3['Position']=='FALSE','F',df_3['Position'])

In [11]:
df_1.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,Allen,C,Brooklyn Nets,2018-10-19,NYK,15,8.693321


In [12]:
df_2.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,Al-Farouq Aminu,F,Portland Trail Blazers,20/10/2018,SAS,14,12.650065


In [13]:
df_3.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,Aldridge,F,San Antonio Spurs,2018-10-20,POR,12,23.908463


### Captalize

In [14]:
def normalize_name(name):
    name = name.lower()
    name = name.capitalize() 
    return name

In [15]:
df_1['Player'] = df_1['Player'].apply(normalize_name)
df_2['Player'] = df_2['Player'].apply(normalize_name)
df_3['Player'] = df_3['Player'].apply(normalize_name)

In [16]:
df_3.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,Aldridge,F,San Antonio Spurs,2018-10-20,POR,12,23.908463


### Double Check Positions

In [17]:
df_1['Position'].unique()

array(['C', nan, 'PG', 'SG'], dtype=object)

In [18]:
df_1[df_1['Position'].isna()]['Player'].nunique()

30

In [19]:
df_1['Team'].nunique()

15

In [20]:
# 15 Teams x 2 Forwards = 30 players (maybe issue with how it was represented) 

In [21]:
df_1[df_1['Position'].isna()]['Player'].unique()

array(['Antetokounmpo', 'Ariza', 'Bogdanovic', 'Bullock', 'Butler',
       'Chandler', 'Collins', 'Dudley', 'Gordon', 'Griffin', 'Harris',
       'Hezonja', 'Isaac', 'J_green', 'Lamb', 'Leonard', 'Love',
       'Markkanen', 'Middleton', 'Morris', 'Osman', 'Porter', 'Prince',
       'Richardson', 'Siakam', 'Tatum', 'Vonleh', 'Williams', 'Winslow',
       'Young'], dtype=object)

    Validated, these are all Forwads

In [22]:
df_1['Position'] = np.where(df_1['Position'].isna(),'F',df_1['Position'])

In [23]:
df_1['Position'].unique()

array(['C', 'F', 'PG', 'SG'], dtype=object)

In [24]:
df_2['Position'].unique()

array(['F', 'SG', 'PG', 'C'], dtype=object)

In [25]:
df_3['Position'].unique()

array(['F', 'PG', 'C', 'SG'], dtype=object)

    Now we need to do a group by for teams

In [26]:
df_1['Team'].nunique()

15

In [27]:
df_1.groupby('Position')['Player'].nunique()

Position
C     15
F     30
PG    15
SG    15
Name: Player, dtype: int64

In [28]:
df_2['Team'].nunique()

5

In [29]:
df_2.groupby('Position')['Player'].nunique()

Position
C      5
F     10
PG     5
SG     5
Name: Player, dtype: int64

In [30]:
df_3['Team'].nunique()

10

In [31]:
df_3.groupby('Position')['Player'].nunique()

Position
C     11
F     22
PG     9
SG     8
Name: Player, dtype: int64

    Issue: Expecting 10 SG, 10 PG, 10 C, and 20 F

In [32]:
# Note: Size is not important, this is just an easy way to see which player is mispositioned  ... using player.csv as ref
df_3.groupby(['Team','Player','Position'])['Player'].size()

Team                   Player        Position
Dallas Mavericks       Barnes        F           150
                       Jordan        C            67
                       Lee           SG           77
                       Matthews      F            67
                       Smith         PG           51
Golden State Warriors  Cousins       C            46
                       Curry         PG           67
                       Durant        F            66
                       Green         F            64
                       Thompson      SG           71
Houston Rockets        Capela        C            65
                       Gordon        F            66
                       Harden        SG           76
                       Paul          PG           56
                       Tucker        F            80
Los Angeles Clippers   Beverley      SG          152
                       Bradley       PG          122
                       Gallinari     F            66


Changes that need to occur

San Antonio Spurs, Forbes - SG

Phoenix Suns, Booker  - SG

New Orleans Pelicans, Moore  - PG

In [33]:
df_3['Position'] = np.where(
    (df_3['Team']=='San Antonio Spurs')
    &
    (df_3['Player']=='Forbes')
, 'SG'
, df_3['Position']
)

In [34]:
df_3['Position'] = np.where(
    (df_3['Team']=='Phoenix Suns')
    &
    (df_3['Player']=='Booker')
, 'SG'
, df_3['Position']
)

In [35]:
df_3['Position'] = np.where(
    (df_3['Team']=='New Orleans Pelicans')
    &
    (df_3['Player']=='Moore')
, 'PG'
, df_3['Position']
)

In [36]:
df_3.groupby('Position')['Player'].nunique()

Position
C     10
F     20
PG    10
SG    10
Name: Player, dtype: int64

### Final Check 5 Players per team

In [37]:
df_1.groupby('Team')['Player'].nunique()

Team
Atlanta Hawks          5
Boston Celtics         5
Brooklyn Nets          5
Charlotte Hornets      5
Chicago Bulls          5
Cleveland Cavaliers    5
Detroit Pistons        5
Indiana Pacers         5
Miami Heat             5
Milwaukee Bucks        5
New York Knicks        5
Orlando Magic          5
Philadelphia 76ers     5
Toronto Raptors        5
Washington Wizards     5
Name: Player, dtype: int64

In [38]:
df_2.groupby('Team')['Player'].nunique()

Team
Denver Nuggets            5
Minnesota Timberwolves    5
Oklahoma City Thunder     5
Portland Trail Blazers    5
Utah Jazz                 5
Name: Player, dtype: int64

In [39]:
df_3.groupby('Team')['Player'].nunique()

Team
Dallas Mavericks         5
Golden State Warriors    5
Houston Rockets          5
Los Angeles Clippers     5
Los Angeles Lakers       5
Memphis Grizzlies        5
New Orleans Pelicans     5
Phoenix Suns             5
Sacramento Kings         5
San Antonio Spurs        5
Name: Player, dtype: int64

## Combine 

In [40]:
master_df = pd.DataFrame()

In [41]:
master_df = master_df.append(df_1)

In [42]:
master_df = master_df.append(df_2)

In [43]:
master_df = master_df.append(df_3)

# Validate
    Number Players
    Number Teams
    Unique Player Names
    Positions by Team
    Nulls

## Correct Teams and Player Count
    Will check uniform team names
    team count
    player count
    another player position distribution

In [44]:
master_df.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,Allen,C,Brooklyn Nets,2018-10-19,NYK,15,8.693321


In [45]:
master_df['Team'].nunique()

30

In [46]:
master_df.groupby('Position')['Player'].nunique()

Position
C     30
F     59
PG    30
SG    30
Name: Player, dtype: int64

In [47]:
# This may be cause by same name
master_df.groupby('Player')['Team'].nunique().reset_index(name='Duplicates').sort_values(by='Duplicates',ascending=False).head(8)

,Player,Duplicates
56,Gordon,2
59,Green,2
80,Johnson,2
101,Morris,1
95,Maurice harkless,1
96,Mcgee,1
97,Mcgruder,1
98,Middleton,1


In [48]:
master_df[master_df['Player']=='Johnson']['Team'].unique()

array(['Miami Heat', 'New Orleans Pelicans'], dtype=object)

In [49]:
master_df['Player'] = np.where(
    (master_df['Team']=='Miami Heat')
    &
    (master_df['Player']=='Johnson')
, 'J_Johnson'
, master_df['Player']
)

In [50]:
master_df['Player'] = np.where(
    (master_df['Team']=='New Orleans Pelicans')
    &
    (master_df['Player']=='Johnson')
, 'W_Johnson'
, master_df['Player']
)

In [51]:
master_df[master_df['Player']=='Gordon']['Team'].unique()

array(['Orlando Magic', 'Houston Rockets'], dtype=object)

In [52]:
master_df['Player'] = np.where(
    (master_df['Team']=='Orlando Magic')
    &
    (master_df['Player']=='Gordon')
, 'A_Gordon'
, master_df['Player']
)

In [53]:
master_df['Player'] = np.where(
    (master_df['Team']=='Houston Rockets')
    &
    (master_df['Player']=='Gordon')
, 'E_Gordon'
, master_df['Player']
)

In [54]:
master_df[master_df['Player']=='Green']['Team'].unique()

array(['Toronto Raptors', 'Golden State Warriors'], dtype=object)

In [55]:
master_df['Player'] = np.where(
    (master_df['Team']=='Toronto Raptors')
    &
    (master_df['Player']=='Green')
, 'D_Green'
, master_df['Player']
)

In [56]:
master_df['Player'] = np.where(
    (master_df['Team']=='Golden State Warriors')
    &
    (master_df['Player']=='Green')
, 'Green'
, master_df['Player']
)

In [57]:
# This may be cause by same name
master_df.groupby('Player')['Team'].nunique().reset_index(name='Duplicates').sort_values(by='Duplicates',ascending=False).head(8)

,Player,Duplicates
0,A_Gordon,1
94,Lowry,1
96,Matthews,1
97,Maurice harkless,1
98,Mcgee,1
99,Mcgruder,1
100,Middleton,1
101,Mirotic,1


In [58]:
master_df['Team'].nunique()

30

In [59]:
master_df.groupby('Position')['Player'].nunique()

Position
C     30
F     60
PG    30
SG    30
Name: Player, dtype: int64

    All good with duplicates and positions 

### Check Nulls 

In [60]:
master_df.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,Allen,C,Brooklyn Nets,2018-10-19,NYK,15,8.693321


In [61]:
master_df.isnull().sum()

Player              0
Position            0
Team                0
Date                0
Opp                 0
Fantasy_Score       0
predictions_2019    0
dtype: int64

# Collect Accuracy

In [62]:
accuracy_df = master_df.copy()

In [63]:
accuracy_df['abs_error'] = abs(accuracy_df['Fantasy_Score']-accuracy_df['predictions_2019'])

In [64]:
accuracy_df['error'] = 100 * (accuracy_df['abs_error']/accuracy_df['Fantasy_Score'])

In [65]:
accuracy_df.shape

(10271, 9)

In [66]:
accuracy_df[accuracy_df['Fantasy_Score']==0].shape

(246, 9)

In [67]:
accuracy_df[accuracy_df['Fantasy_Score']==0].head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019,abs_error,error
386,Batum,SG,Charlotte Hornets,2018-11-17,PHI,0,17.563246,17.563246,inf


In [68]:
accuracy_df['error'] = np.where(accuracy_df['Fantasy_Score']==0,accuracy_df['abs_error'],accuracy_df['error'])

In [69]:
# accuracy_df = accuracy_df[accuracy_df['Fantasy_Score']!=0]

In [70]:
accuracy_df['Accuracy'] = 100 - accuracy_df['error']

In [71]:
accuracy_df = accuracy_df[accuracy_df['Fantasy_Score']!=0]

In [72]:
np.mean(np.abs((accuracy_df['Fantasy_Score'] - accuracy_df['predictions_2019']) / accuracy_df['Fantasy_Score'])) * 100

64.235605486037

In [73]:
np.median(np.abs((accuracy_df['Fantasy_Score'] - accuracy_df['predictions_2019']) / accuracy_df['Fantasy_Score'])) * 100

33.974745469894

    MdAPE 

In [74]:
np.median(np.abs((accuracy_df['Fantasy_Score'] - accuracy_df['predictions_2019']) / accuracy_df['Fantasy_Score'])) * 100

33.974745469894

    MAPE

In [75]:
np.mean(np.abs((accuracy_df['Fantasy_Score'] - accuracy_df['predictions_2019']) / accuracy_df['Fantasy_Score'])) * 100

64.235605486037

    Mean Absolute Error 

In [76]:
accuracy_df['abs_error'].mean()

7.218469198904643

In [77]:
from sklearn.metrics import r2_score

In [78]:
r2_score(accuracy_df['Fantasy_Score'], accuracy_df['predictions_2019'])

0.25991292575708036

In [79]:
from sklearn.metrics import mean_squared_error
import math


In [80]:
print(mean_squared_error(accuracy_df['Fantasy_Score'], accuracy_df['predictions_2019']))
print(math.sqrt(mean_squared_error(accuracy_df['Fantasy_Score'], accuracy_df['predictions_2019'])))

87.5825681388802
9.358555878920647


In [81]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(accuracy_df['Fantasy_Score'], accuracy_df['predictions_2019']))

7.218469198904643


# Run Simulations

In [82]:
master_df.to_excel('Final_Master.xlsx')

In [83]:
master_df.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019
0,Allen,C,Brooklyn Nets,2018-10-19,NYK,15,8.693321


In [84]:
master_df.groupby(['Team','Opp'])['Date'].nunique().reset_index().head(10)

,Team,Opp,Date
0,Atlanta Hawks,BOS,4
1,Atlanta Hawks,BRK,3
2,Atlanta Hawks,CHI,4
3,Atlanta Hawks,CHO,4
4,Atlanta Hawks,CLE,3
5,Atlanta Hawks,DAL,2
6,Atlanta Hawks,DEN,2
7,Atlanta Hawks,DET,3
8,Atlanta Hawks,GSW,2
9,Atlanta Hawks,HOU,2


In [85]:
team_map = {
    "BRK":'Brooklyn Nets',
    "MIL":'Milwaukee Bucks',
    "WAS":'Washington Wizards',
    "ORL":'Orlando Magic',
    "CHO":'Charlotte Hornets',
    "ATL":'Atlanta Hawks',
    "IND":'Indiana Pacers',
    "DET":'Detroit Pistons',
    "PHI":'Philadelphia 76ers',
    "CHI":'Chicago Bulls',
    "TOR":'Toronto Raptors',
    "NYK":'New York Knicks',
    "CLE":'Cleveland Cavaliers',
    "BOS":'Boston Celtics',
    "MIA":'Miami Heat',
    "POR":'Portland Trail Blazers',
    "MIN":'Minnesota Timberwolves',
    "UTA":'Utah Jazz',
    "DEN":'Denver Nuggets',
    "OKC":'Oklahoma City Thunder',
    "SAS":'San Antonio Spurs',
    "MEM":'Memphis Grizzlies',
    "LAL":'Los Angeles Lakers',
    "DAL":'Dallas Mavericks',
    "PHO":'Phoenix Suns',
    "LAC":'Los Angeles Clippers',
    "SAC":'Sacramento Kings',
    "HOU":'Houston Rockets',
    "GSW":'Golden State Warriors',
    "NOP":'New Orleans Pelicans'
}

In [86]:
master_df['Opp'] = master_df['Opp'].map(team_map)

In [87]:
master_df.groupby(['Team','Opp'])['Date'].nunique().reset_index().head(10)

,Team,Opp,Date
0,Atlanta Hawks,Boston Celtics,4
1,Atlanta Hawks,Brooklyn Nets,3
2,Atlanta Hawks,Charlotte Hornets,4
3,Atlanta Hawks,Chicago Bulls,4
4,Atlanta Hawks,Cleveland Cavaliers,3
5,Atlanta Hawks,Dallas Mavericks,2
6,Atlanta Hawks,Denver Nuggets,2
7,Atlanta Hawks,Detroit Pistons,3
8,Atlanta Hawks,Golden State Warriors,2
9,Atlanta Hawks,Houston Rockets,2


In [88]:
def make_game_key(vec):
    team = vec[0]
    opp = vec[1]
    date = vec[2]
    
    normalized_teams = sorted([team,opp])
    
    key = str(normalized_teams)+str(date)
    
    return key

In [89]:
master_df['Game_Key'] = master_df[['Team','Opp','Date']].apply(make_game_key,axis=1)

In [90]:
filter_df = master_df.groupby('Game_Key')['Player'].nunique().reset_index(name='Player_Count')

In [91]:
filter_df['Keep'] = np.where(filter_df['Player_Count']>=8,True,False)

In [92]:
# Number of games (according to our data)
filter_df.shape

(2421, 3)

In [93]:
filter_df.groupby('Keep').size().reset_index()

,Keep,0
0,False,2022
1,True,399


In [94]:
# For Now, we will need to use only the True Games
filter_df = filter_df[filter_df['Keep']==True]

In [95]:
filter_df.head()

,Game_Key,Player_Count,Keep
0,"['Atlanta Hawks', 'Boston Celtics']2018-11-23",9,True
1,"['Atlanta Hawks', 'Boston Celtics']2018-12-14",8,True
2,"['Atlanta Hawks', 'Boston Celtics']2019-01-19",9,True
3,"['Atlanta Hawks', 'Boston Celtics']2019-03-16",10,True
4,"['Atlanta Hawks', 'Brooklyn Nets']2018-12-16",8,True


In [96]:
master_df.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019,Game_Key
0,Allen,C,Brooklyn Nets,2018-10-19,New York Knicks,15,8.693321,"['Brooklyn Nets', 'New York Knicks']2018-10-19"


In [97]:
master_df.shape

(10271, 8)

In [98]:
master_df = master_df[master_df['Game_Key'].isin(filter_df['Game_Key'].unique())]

In [99]:
master_df.shape

(3608, 8)

In [104]:
master_df.to_excel('For_Dashboard_Master.xlsx')

In [105]:
master_df.head(1)

,Player,Position,Team,Date,Opp,Fantasy_Score,predictions_2019,Game_Key
0,Allen,C,Brooklyn Nets,2018-10-19,New York Knicks,15,8.693321,"['Brooklyn Nets', 'New York Knicks']2018-10-19"


# Simulate 

In [132]:
def find_percent(df):
    team_usage = {}
    for team in df['Team'].unique():
        team_usage[team] = 0
    positions = {
        "C":0,"SG":0,"PG":0,"F":0
    }
    
    picked_players = []
    
    df = df.sort_values(by='predictions_2019',ascending=False)
    for index, row in df.iterrows():
        player = row['Player']
        position = row['Position']
        team = row['Team']
        
        if positions[position]<1:
            pass
        elif (position=='F')&(positions[position]<2):
            pass
        else:
            continue 
            
        if team_usage[team] >= 3:
            continue
            
        # Mark that we are using this player
        positions[position] += 1
        team_usage[team] += 1
        
        picked_players.append(player)
        
    # ---------
        
    team_usage = {}
    for team in df['Team'].unique():
        team_usage[team] = 0
    positions = {
        "C":0,"SG":0,"PG":0,"F":0
    }
    
    best_players = []
    
    df = df.sort_values(by='Fantasy_Score',ascending=False)
    for index, row in df.iterrows():
        player = row['Player']
        position = row['Position']
        team = row['Team']
        
        if positions[position]<1:
            pass
        elif (position=='F')&(positions[position]<2):
            pass
        else:
            continue 
            
        if team_usage[team] >= 3:
            continue
            
        # Mark that we are using this player
        positions[position] += 1
        team_usage[team] += 1
        
        best_players.append(player)
    
    print("Picks:  ",picked_players)
    print("Optimal:",best_players)
    
    delta = list(set(best_players)-set(picked_players))
    
    print("Delta:  ",delta)
    
    Percent_Correct = 100-(len(delta)*20)
    
    print("Percent Correct: ",Percent_Correct,"%")
    
    return Percent_Correct

In [133]:
percents = []
for game in master_df['Game_Key'].unique():
    tmp_df = master_df[master_df['Game_Key']==game]
    x = find_percent(tmp_df)
    percents.append(x)

Picks:   ['Russell', 'Hardaway', 'Hezonja', 'Kanter', 'Harris']
Optimal: ['Hardaway', 'Kanter', 'Russell', 'Harris', 'Dudley']
Delta:   ['Dudley']
Percent Correct:  80 %
Picks:   ['Oladipo', 'Turner', 'Russell', 'Bogdanovic', 'Harris']
Optimal: ['Oladipo', 'Harris', 'Bogdanovic', 'Collison', 'Allen']
Delta:   ['Collison', 'Allen']
Percent Correct:  60 %
Picks:   ['Russell', 'Love', 'Harris', 'Hood', 'Allen']
Optimal: ['Harris', 'Russell', 'Dudley', 'Hood', 'Zizic']
Delta:   ['Dudley', 'Zizic']
Percent Correct:  60 %
Picks:   ['Davis', 'Holiday', 'Mirotic', 'Russell', 'Harris']
Optimal: ['Russell', 'Holiday', 'Davis', 'Dudley', 'Mirotic']
Delta:   ['Dudley']
Percent Correct:  80 %
Picks:   ['Hardaway', 'Kanter', 'Russell', 'Hezonja', 'Harris']
Optimal: ['Hardaway', 'Russell', 'Harris', 'Kanter', 'Hezonja']
Delta:   []
Percent Correct:  100 %
Picks:   ['Griffin', 'Russell', 'Bullock', 'Drummond']
Optimal: ['Griffin', 'Harris', 'Jackson', 'Drummond']
Delta:   ['Jackson', 'Harris']
Percent

Picks:   ['Beal', 'Griffin', 'Bullock', 'Drummond', 'Satoransky']
Optimal: ['Griffin', 'Ariza', 'Jackson', 'Beal', 'Drummond']
Delta:   ['Jackson', 'Ariza']
Percent Correct:  60 %
Picks:   ['Lavine', 'Dunn', 'Markkanen', 'Ariza', 'Bryant']
Optimal: ['Lavine', 'Markkanen', 'Dunn', 'Ariza', 'Bryant']
Delta:   []
Percent Correct:  100 %
Picks:   ['Beal', 'Lamb', 'J_green', 'Satoransky', 'Zeller']
Optimal: ['Ariza', 'Satoransky', 'Williams', 'Beal', 'Zeller']
Delta:   ['Williams', 'Ariza']
Percent Correct:  60 %
Picks:   ['Beal', 'Ariza', 'J_green', 'Dedmon', 'Carter']
Optimal: ['Beal', 'J_green', 'Collins', 'Satoransky', 'Dedmon']
Delta:   ['Collins', 'Satoransky']
Percent Correct:  60 %
Picks:   ['Beal', 'Richardson', 'Whiteside', 'Ariza', 'J_Johnson']
Optimal: ['Beal', 'Ariza', 'J_Johnson', 'Richardson', 'Whiteside']
Delta:   []
Percent Correct:  100 %
Picks:   ['Beal', 'Butler', 'Embiid', 'Simmons', 'Ariza']
Optimal: ['Beal', 'Embiid', 'Butler', 'J_green', 'Simmons']
Delta:   ['J_green

Picks:   ['Walker', 'Griffin', 'Bullock', 'Batum', 'Drummond']
Optimal: ['Williams', 'Walker', 'Griffin', 'Drummond', 'Batum']
Delta:   ['Williams']
Percent Correct:  80 %
Picks:   ['Irving', 'Lamb', 'Tatum', 'Batum', 'Horford']
Optimal: ['Irving', 'Tatum', 'Lamb', 'Horford', 'Batum']
Delta:   []
Percent Correct:  100 %
Picks:   ['Walker', 'Fournier', 'A_Gordon', 'Vucevic', 'Lamb']
Optimal: ['Walker', 'A_Gordon', 'Vucevic', 'Williams', 'Fournier']
Delta:   ['Williams']
Percent Correct:  80 %
Picks:   ['Walker', 'Lamb', 'Williams', 'Cauley-stein']
Optimal: ['Walker', 'Batum', 'Cauley-stein', 'Lamb', 'Shumpert']
Delta:   ['Batum', 'Shumpert']
Percent Correct:  60 %
Picks:   ['Lamb', 'Batum', 'Fox', 'Cauley-stein', 'Williams']
Optimal: ['Hield', 'Lamb', 'Williams', 'Fox', 'Cauley-stein']
Delta:   ['Hield']
Percent Correct:  80 %
Picks:   ['Walker', 'Booker', 'Warren', 'Lamb', 'Bender']
Optimal: ['Booker', 'Oubre', 'Lamb', 'Walker', 'Bender']
Delta:   ['Oubre']
Percent Correct:  80 %
Picks

Picks:   ['Butler', 'Embiid', 'Redick', 'Conley', 'Anderson']
Optimal: ['Redick', 'Conley', 'Butler', 'Gasol', 'Anderson']
Delta:   ['Gasol']
Percent Correct:  80 %
Picks:   ['Embiid', 'Leonard', 'Redick', 'Lowry', 'Butler']
Optimal: ['Leonard', 'Butler', 'Redick', 'Ibaka', 'Lowry']
Delta:   ['Ibaka']
Percent Correct:  80 %
Picks:   ['Embiid', 'Butler', 'Chandler', 'Hood']
Optimal: ['Butler', 'Embiid', 'Simmons', 'Osman', 'Hood']
Delta:   ['Osman', 'Simmons']
Percent Correct:  60 %
Picks:   ['Derozan', 'Butler', 'Redick', 'Chandler', 'White']
Optimal: ['Gay', 'Derozan', 'Redick', 'Aldridge', 'Simmons']
Delta:   ['Gay', 'Simmons', 'Aldridge']
Percent Correct:  40 %
Picks:   ['Embiid', 'Butler', 'Redick', 'Hezonja', 'Mudiay']
Optimal: ['Hardaway', 'Embiid', 'Butler', 'Simmons', 'Vonleh']
Delta:   ['Hardaway', 'Vonleh', 'Simmons']
Percent Correct:  40 %
Picks:   ['Irving', 'Embiid', 'Redick', 'Butler', 'Morris']
Optimal: ['Irving', 'Embiid', 'Butler', 'Morris', 'Redick']
Delta:   []
Perce

Picks:   ['Damian lillard', 'Jusuf nurkic', 'Paul george', 'Cj mccollum', 'Jerami grant']
Optimal: ['Cj mccollum', 'Paul george', 'Damian lillard', 'Jusuf nurkic', 'Jerami grant']
Delta:   []
Percent Correct:  100 %
Picks:   ['Damian lillard', 'Jusuf nurkic', 'Cj mccollum', 'Joe ingles', 'Derrick favors']
Optimal: ['Damian lillard', 'Jusuf nurkic', 'Cj mccollum', 'Joe ingles', 'Derrick favors']
Delta:   []
Percent Correct:  100 %
Picks:   ['Damian lillard', 'Jusuf nurkic', 'Paul george', 'Cj mccollum']
Optimal: ['Paul george', 'Damian lillard', 'Jusuf nurkic', 'Al-farouq aminu', 'Terrance ferguson']
Delta:   ['Al-farouq aminu', 'Terrance ferguson']
Percent Correct:  60 %
Picks:   ['Damian lillard', 'Jusuf nurkic', 'Paul george', 'Cj mccollum', 'Jerami grant']
Optimal: ['Damian lillard', 'Jusuf nurkic', 'Paul george', 'Cj mccollum', 'Jerami grant']
Delta:   []
Percent Correct:  100 %
Picks:   ['Damian lillard', 'Jusuf nurkic', 'Andrew wiggins', 'Dario saric', 'Jerryd bayless']
Optimal: 

Picks:   ['Davis', 'Mirotic', 'Hield', 'Moore', 'Bjelica']
Optimal: ['Mirotic', 'Davis', 'Hield', 'Fox', 'Shumpert']
Delta:   ['Fox', 'Shumpert']
Percent Correct:  60 %
Picks:   ['Harden', 'E_Gordon', 'Fox', 'Capela', 'Bjelica']
Optimal: ['Harden', 'Fox', 'Capela', 'E_Gordon', 'Shumpert']
Delta:   ['Shumpert']
Percent Correct:  80 %
Picks:   ['Davis', 'Holiday', 'Moore', 'Bjelica', 'Bjelica']
Optimal: ['Hield', 'Davis', 'Fox', 'Shumpert']
Delta:   ['Fox', 'Hield', 'Shumpert']
Percent Correct:  40 %
Picks:   ['Conley', 'Gasol', 'Hield', 'Bjelica', 'Bjelica']
Optimal: ['Hield', 'Bjelica', 'Bjelica', 'Conley', 'Gasol']
Delta:   []
Percent Correct:  100 %


In [138]:
percents_df = pd.DataFrame(percents,columns=['Percent_Correct'])

In [139]:
percents_df.head(2)

,Percent_Correct
0,80
1,60


In [143]:
grouped_percents = percents_df.groupby('Percent_Correct').size().reset_index(name='Count').sort_values(by='Count',ascending='False')

In [144]:
grouped_percents

,Percent_Correct,Count
0,20,9
1,40,36
4,100,104
2,60,124
3,80,126


In [145]:
grouped_percents['Percent_Correct'] = grouped_percents['Percent_Correct'].astype(str) + "%"

In [146]:
grouped_percents

,Percent_Correct,Count
0,20%,9
1,40%,36
4,100%,104
2,60%,124
3,80%,126


In [147]:
grouped_percents['Percent_of_Games'] = round((grouped_percents['Count']/grouped_percents['Count'].sum())*100,2)

In [148]:
grouped_percents

,Percent_Correct,Count,Percent_of_Games
0,20%,9,2.26
1,40%,36,9.02
4,100%,104,26.07
2,60%,124,31.08
3,80%,126,31.58
